In [1]:
import numpy as np
import pandas as pd
from functools import partial
from collections import Counter
import re

### Make sure that the below files are uploaded in the temorary runtime in Google Collab or in the  same directory as the notebook if you are running via Jupyter.

You can change the variables 'train_file' and 'test_file' with your own file names which should be in the csv format <PC,Memory address,LoadValue> or the original format eg: 0x7f64b4765e1f: R 0x7f64b4791e68, 0x000000000000000e

In [2]:
train_file = 'train_pinatrace.out'
test_file = 'test_pinatrace.out'

In [3]:
def csv_format(fname):

    with open(fname, 'r+') as f:
        text = f.read()
        text = re.sub(' ','',text)
        text = re.sub(':',',',text)
        text = re.sub('R','', text)
        text = re.sub('#eof','',text)

        f.seek(0)
        f.write(text)
        f.truncate()
        
def map(Y):
    mapping={v:k for k,v in enumerate(set(Y))}
    Y1=[mapping[y] for y in Y]
    return np.array(Y1)

def get_data(fname):
    csv_format(fname)
    df = pd.read_csv(fname, sep=",", header=None, names=["PC", "MemAdd", "Val"], converters={0:partial(int, base=16),
                                                                                            1:partial(int, base=16),
                                                                                            2:partial(int, base=16)})
    p,m,v = df.nunique()
    PC = df['PC'].to_numpy()
    MemAdd = df['MemAdd'].to_numpy()
    Val = df['Val'].to_numpy()
    PC = map(list(PC))
    MemAdd = map(list(MemAdd))
    return PC, MemAdd, Val, p, m

In [4]:
train_PC, train_MemAdd, train_Val, train_unique_PCs, train_unique_memadds = get_data(train_file)

test_PC, test_MemAdd, test_Val, test_unique_PCs, test_unique_memadds = get_data(test_file)


#  Baseline : Last Value Predictor

In [5]:

def baseline_predictor(PC, MemAdd, Val, p, m):
    correct = 0
    total_vals = Val.shape[0]
    p_dic = {}
            
    for i in range(total_vals):
        if (PC[i], MemAdd[i]) not in p_dic.keys():
            p_dic[(PC[i], MemAdd[i])] = 0
        if p_dic[(PC[i], MemAdd[i])] == Val[i]:
            correct +=1
        else:
            p_dic[(PC[i], MemAdd[i])] = Val[i]
                
    return (correct/total_vals)*100

In [6]:
train_accu = baseline_predictor(train_PC, train_MemAdd, train_Val, train_unique_PCs, train_unique_memadds)
print("The train accuracy for baseline Last Value predictor is {:.2f}".format(train_accu))

The train accuracy for baseline Last Value predictor is 63.35


In [7]:
test_accu = baseline_predictor(test_PC, test_MemAdd, test_Val, test_unique_PCs, test_unique_memadds)
print("The test accuracy for baseline Last Value predictor is {:.2f}".format(test_accu))

The test accuracy for baseline Last Value predictor is 64.08


## Predictor for Last n load values

In [8]:
def last_n_vals(PC, MemAdd, Val, p, m, total_vals,n):
    correct = 0
    p_dic = {}
    
    for i in range(total_vals):
        if PC[i] not in p_dic.keys():
            p_dic[PC[i]] = np.zeros((n), dtype = 'int64')
        current_vec = p_dic[PC[i]]
        pred_val = Counter(current_vec).most_common(1)[0][0]
        if pred_val == Val[i]:
            correct +=1
        else:
            current_vec = current_vec[1:]
            current_vec = np.append(current_vec, int(Val[i]))
            p_dic[PC[i]] = current_vec
    return (correct/total_vals)*100
        

In [9]:
train_accu = last_n_vals(train_PC, train_MemAdd, train_Val, train_unique_PCs, train_unique_memadds, train_Val.shape[0], 5)
print("The train accuracy for Global Load Value Predictor is {:.2f}".format(train_accu))

The train accuracy for Global Load Value Predictor is 47.62


In [10]:
test_accu = last_n_vals(test_PC, test_MemAdd, test_Val, test_unique_PCs, test_unique_memadds, test_Val.shape[0], 5)
print("The test accuracy for Global Load Value Predictor is {:.2f}".format(test_accu))

The test accuracy for Global Load Value Predictor is 48.53


## Predictor for Last n load values at perticular Memory Address

In [11]:
def last_n_mem_vals(PC, MemAdd, Val, p, m, total_vals,n):
    correct = 0
    p_dic = {}
    
    for i in range(total_vals):
        if (PC[i], MemAdd[i]) not in p_dic.keys():
            p_dic[(PC[i], MemAdd[i])] = np.zeros((n), dtype = 'int64')
        current_vec = p_dic[(PC[i], MemAdd[i])]
        pred_val = Counter(current_vec).most_common(1)[0][0]
        if pred_val == Val[i]:
            correct +=1
        else:
            current_vec = current_vec[1:]
            current_vec = np.append(current_vec, int(Val[i]))
            p_dic[(PC[i], MemAdd[i])] = current_vec
        
    return (correct/total_vals)*100


In [12]:
train_accu = last_n_mem_vals(train_PC, train_MemAdd, train_Val, train_unique_PCs, train_unique_memadds, train_Val.shape[0], 5)
print("The train accuracy for Local Load Value Predictor {:.2f}".format(train_accu))

The train accuracy for Local Load Value Predictor 55.95


In [13]:
test_accu = last_n_mem_vals(test_PC, test_MemAdd, test_Val, test_unique_PCs, test_unique_memadds, test_Val.shape[0], 5)
print("The test accuracy for Local Load Value Predictor {:.2f}".format(test_accu))

The test accuracy for Local Load Value Predictor 56.94


# Removing unpredictable load

In [14]:
def unpredictable_loads(PC, MemAdd, Val, p, m, total_vals):
    
    max_checks = int(total_vals/p)
    correct = 0
    unpredictable_load_lst = []
    skipped = 0
    already_pred  = 0
    p_dic = {}
    p_lval_table = np.zeros((p,m), dtype = 'uint64')
    for i in range(total_vals):
        if PC[i] in unpredictable_load_lst:
            skipped+=1
            continue
        else:
            if PC[i] not in p_dic.keys():
                p_dic[PC[i]] = [1]
            if p_lval_table[PC[i]][MemAdd[i]] == Val[i]:
                correct +=1
                p_dic[PC[i]].append(1)
            else:
                p_lval_table[PC[i]][MemAdd[i]] = Val[i]
                p_dic[PC[i]].append(0)
            if p_dic[PC[i]].count(0) > max_checks:
                unpredictable_load_lst.append(PC[i])
                already_pred += len(p_dic[PC[i]])
                
    return (correct/(total_vals-skipped+already_pred))*100
        

In [15]:
train_accu = unpredictable_loads(train_PC, train_MemAdd, train_Val, train_unique_PCs, train_unique_memadds, train_Val.shape[0])
print("The train accuracy for Predictable Loads Last Value Predictor  is {:.2f}".format(train_accu))

The train accuracy for Predictable Loads Last Value Predictor  is 82.87


In [16]:
test_accu = unpredictable_loads(test_PC, test_MemAdd, test_Val, test_unique_PCs, test_unique_memadds, test_Val.shape[0])
print("The test accuracy for Predictable Loads Last Value Predictor is {:.2f}".format(test_accu))

The test accuracy for Predictable Loads Last Value Predictor is 82.92
